<a href="https://colab.research.google.com/github/CalvHobbes/shop_india/blob/main/smolagents/price_comp_smolagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install smolagents smolagents[litellm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [10]:
from google.colab import userdata
import os

openapi_key = userdata.get('OPENAPI_KEY')
hf_token = userdata.get('HF_TOKEN')

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py

--2025-01-31 12:14:08--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1811 (1.8K) [text/plain]
Saving to: ‘search_Croma.py’

search_Croma.py     100%[===================>]   1.77K  --.-KB/s    in 0s      

2025-01-31 12:14:09 (28.8 MB/s) - ‘search_Croma.py’ saved [1811/1811]

--2025-01-31 12:14:09--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1460

In [11]:
model_id = "gpt-4o-mini"

In [12]:
from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(model_id=model_id, api_key=openapi_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [23]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [20]:
from smolagents import tool
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance

@tool
def search_croma_tool(product: str) -> str:
    """
    This is a tool that searches for the user specified product on Croma and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_croma(product)


@tool
def search_reliance_tool(product: str) -> str:
    """
    This is a tool that searches for the user specified product on Reliance and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_reliance(product)


In [28]:
from smolagents.prompts import CODE_SYSTEM_PROMPT
modified_system_prompt = f"{CODE_SYSTEM_PROMPT}\n{instructions}"
agent = CodeAgent(tools=[search_croma_tool,search_reliance_tool], model=model, system_prompt=modified_system_prompt, additional_authorized_imports=['requests', 'bs4', 'json'])
agent.run("iphone 16 pro max")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ iphone 16 pro max                                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  croma_results = search_croma_tool(product="iphone 16 pro max")                                                   
  print("Croma results:", croma_results)                                                                           
                                                                                                                   
  reliance_results = search_reliance_tool(product="iphone 16 pro max")                                             
  print("Reliance results:", reliance_results)                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Croma results: [{'name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774', 'price': 177900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (512GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (1TB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-white-titanium-/p/309771', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (1TB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-desert-titanium-/p/309773', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (512GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-desert-titanium-/p/309731', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-desert-titanium-/p/309727', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-black-titanium-/p/309729', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-natural-titanium-/p/309732', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-natural-titanium-/p/309728', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-black-titanium-/p/309725', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-white-titanium-/p/309730', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-white-titanium-/p/309726', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-512gb-black-titanium-/p/309733', 'price': 142900.0}]
Reliance results: [{'name': 'Apple iPhone 16 Pro Max 1 TB, Desert Titanium', 'url': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-desert-titanium/p/494423069', 'price': '1,77,900.00'},
{'name': 'Apple iPhone 16 Pro Max 256 GB, Black Titanium', 'url': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059', 'price': 
'1,37,900.00'}, {'name': 'Apple iPhone 16 Pro Max 512 GB, White Titanium', 'url': 
'https://www.reliancedigital.i

[Step 0: Duration 7.26 seconds| Input tokens: 2,412 | Output tokens: 107]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_output = "Product Name: iPhone 16 Pro Max\n"                                                               
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746], Price: ₹137900.0\n"             
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747], Price: ₹137900.0\n"            
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742], Price: ₹137900.0\n"              
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744], Price: ₹137900.0\n"              
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059], Price: ₹137900.0\n"  
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060], Price: ₹137900.0\n"  
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061], Price:              
  ₹137900.0\n"                                                                                                     
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-natural-titanium/p/494423062], Price:             
  ₹137900.0\n"                                                                                                     
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748], Price: ₹157900.0\n"              
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751], Price: ₹157900.0\n"            
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750], Price: ₹157900.0\n"             
  final_output += "- Store: Croma, Product URL:                                                                    
  [https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749], Price: ₹157900.0\n"              
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-white-titanium/p/494423064], Price: ₹157900.0\n"  
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-desert-titanium/p/494423065], Price:              
  ₹157900.0\n"                                                                                                     
  final_output += "- Store: Reliance, Product URL:                                                                 
  [https://www.reliancedigital.in/apple-iphone-16-pro-max

Out - Final answer: Product Name: iPhone 16 Pro Max
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746], 
Price: ₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747], 
Price: ₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742], Price:
₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744], Price:
₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059], Price: ₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060], Price: ₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061], Price: ₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-natural-titanium/p/494423062], Price: ₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748], Price:
₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751], 
Price: ₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750], 
Price: ₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749], Price:
₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-white-titanium/p/494423064], Price: ₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-desert-titanium/p/494423065], Price: ₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-black-titanium/p/494423063], Price: ₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-natural-titanium/p/494423066], Price: ₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754], Price: 
₹177900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774], Price:
₹177900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-white-titanium-/p/309771], Price: 
₹177900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-desert-titanium-/p/309773], Price: 
₹177900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-desert-titanium/p/494423069], Price: ₹177900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-black-titanium/p/494423067], Price: ₹177900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-white-titanium/p/494423068], Price: ₹177900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-natural-titanium/p/494423070], Price: ₹177900.0

[Step 2: Duration 23.16 seconds| Input tokens: 17,684 | Output tokens: 3,030]

'Product Name: iPhone 16 Pro Max\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746], Price: ₹137900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747], Price: ₹137900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742], Price: ₹137900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744], Price: ₹137900.0\n- Store: Reliance, Product URL: [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059], Price: ₹137900.0\n- Store: Reliance, Product URL: [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060], Price: ₹137900.0\n- Store: Reliance, Product URL: [https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061], Price: ₹137900.0\n- Store: Reliance, Product U